In [1]:
from functions.filtrar_datos_real_vert import filtrar_datos_vertical
from functions.PitotProcess import PitotProcess
import pandas as pd
import numpy as np

In [2]:
# Ruta del archivo Excel
archivo_excel = "prueba_tunel_2025-01-15_15-57-53.xlsx"
# Leer el archivo Excel y convertirlo en un dataset
# Por defecto, lee la primera hoja del archivo
dataset = pd.read_excel(archivo_excel)

In [8]:

dataset = dataset.iloc[:,:8]#se toman solo las 8 columnas de los datos tomados
datatemp = pd.DataFrame(columns=dataset.columns)#datafrae temporal para el datahistory
datatemp = dataset.head(34)#los primeros 34 valores de la medicion se nulifican para definir el filtro

In [9]:
database_pitots = pd.DataFrame(columns=["V1","V2", "V3", "V4", "COUPLE 1", "COUPLE 2"]) #creacion de database para guardar datos de cada pitot

In [10]:
local_pressure = 101325 #esto es la presion atmosferica en Pa. Se puso ahí 1 atm
euler_angles = [0,0,0] #angulos de euler dados por IMU anulados por pruebas estaticas
offset_pitots = [0, 0, 0, 0] #Valores de los offset de cada sensor de presión. primero se le resta el offset y luego se filtran los datos

for i in range(len(dataset)):
    local_temp = datatemp[['Temperatura_1','Temperatura_2','Temperatura_3','Temperatura_4']].mean().mean()
    new_Value = dataset.loc[i]

    filtered_data, datatemp = filtrar_datos_vertical(new_Value,datatemp, offset_pitots)
    pressures_data = filtered_data[0:4]

    rho = (local_pressure*0.02897)/(8.314472*(local_temp+273.15))
    velocity_pitots = list(map(lambda x: ((2*abs(x)/rho)**0.5), pressures_data)) #se calcula la velocidad de cada pitot con la presion filtrada                              
    
    proccessed_pitot_couples = PitotProcess(pressures_data,local_pressure,local_temp,euler_angles)
    database_pitots.loc[i] = [velocity_pitots[0], velocity_pitots[1], velocity_pitots[2], velocity_pitots[3], proccessed_pitot_couples[0], proccessed_pitot_couples[1]]
    #print(filtered_data-new_Value)

print(rho)

/home/olga/Escritorio/Modelo matematico/Modelo-Matematico/simulacion/functions/filtrar_datos_real_vert.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_history.iloc[1:, i] = data_history.iloc[:-1, i].values  # Desplazar hacia abajo
/home/olga/Escritorio/Modelo matematico/Modelo-Matematico/simulacion/functions/filtrar_datos_real_vert.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_history.iloc[0, i] = dato  # Insertar el nuevo dato en la primera fila
/home/olga/Escritorio/Modelo matematico/Modelo-Matematico/simulacion/functions/filtrar_datos_real_vert.py:29: SettingWithCopyWarning: 
A value is trying 

1.1765531797319406


In [11]:
database_pitots #RESULTADO FINAL, VELOCIDAD MEDIDA EN CADA PITOT Y EN LAS DOS PAREJAS

,V1,V2,V3,V4,COUPLE 1,COUPLE 2
0,2.609322,1.822874,1.167577,0.799505,"[2.60932246007311, 1.8228736431386006, 0.0]","[1.1675766330067208, 0.7995053637220013, 0.0]"
1,2.620223,1.595820,0.099844,1.268488,"[2.6202230894532526, 1.5958199535079076, 0.0]","[0.09984395261145569, 1.2684881670314514, 0.0]"
2,2.449416,2.010486,1.152017,1.471581,"[2.4494158841203526, 2.010485643047729, 0.0]","[1.152017385275278, 1.4715814522301027, 0.0]"
3,2.631262,1.940330,0.909804,1.357086,"[2.631261540980251, 1.9403303785845452, 0.0]","[0.9098041784941852, 1.357086490645053, 0.0]"
4,2.548004,2.099199,0.864130,1.029677,"[2.5480041794114445, 2.099199390829611, 0.0]","[0.8641295010360925, 1.0296770031717315, 0.0]"
...,...,...,...,...,...,...
504,2.220869,2.002011,1.132483,1.356321,"[2.220868671469014, 2.0020108458160752, 0.0]","[1.1324830736076568, 1.3563212385603682, 0.0]"
505,2.176312,1.699617,0.772916,1.466632,"[2.1763115082595306, 1.6996165866366149, 0.0]","[0.7729156560214794, 1.4666317328125749, 0.0]"
506,2.351494,2.135278,1.034586,1.167057,"[2.351494056331976, 2.135277674767307, 0.0]","[1.0345862643883739, 1.167057439217224, 0.0]"
507,2.512154,1.740058,1.673054,1.498141,"[2.5121539143301237, 1.7400575297956977, 0.0]","[1.6730540037153918, 1.498140987766418, 0.0]"
